In [22]:
import pandas as pd
import numpy as np
import pickle

In [23]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [24]:
PROFILE_DIR = "/mnt/storage/research/princeton/resource_estimation/gemm/3090/" 

In [25]:
with open(PROFILE_DIR + "profiling_df.pickle", "rb") as in_file:
    df = pickle.load(in_file)

In [26]:
df

,M,K,N,Kernel Name,Block Size,Grid Size,DRAM Frequency (cycle/second),SM Frequency (cycle/second),Elapsed Cycles (cycle),Memory Throughput (%),DRAM Throughput (%),Duration (nsecond),L1/TEX Cache Throughput (%),NaN,L2 Cache Throughput (%),SM Active Cycles (cycle),Compute (SM) Throughput (%),Memory [%] (%),Compute (SM) [%] (%),SM: Pipe Tensor Cycles Active (%),SM: Inst Executed Pipe Lsu (%),SM: Issue Active (%),SM: Inst Executed (%),SM: Mio2rf Writeback Active (%),SM: Pipe Alu Cycles Active (%),SM: Mio Inst Issued (%),SM: Inst Executed Pipe Uniform (%),SM: Mio Pq Read Cycles Active (%),SM: Mio Pq Write Cycles Active (%),SM: Pipe Fmaheavy Cycles Active (%),SM: Inst Executed Pipe Adu (%),SM: Pipe Fma Cycles Active (%),SM: Inst Executed Pipe Cbu Pred On Any (%),IDC: Request Cycles Active (%),SM: Inst Executed Pipe Ipa (%),SM: Inst Executed Pipe Tex (%),SM: Inst Executed Pipe Xu (%),SM: Pipe Fp64 Cycles Active (%),L2: T Sectors (%),L2: Lts2xbar Cycles Active (%),L2: Xbar2lts Cycles Active (%),DRAM: Cycles Active (%),L1: M Xbar2l1tex Read Sectors (%),L2: T Tag Requests (%),DRAM: Dram Sectors (%),L1: Data Pipe Lsu Wavefronts (%),L1: Lsu Writeback Active (%),L1: M L1tex2xbar Req Cycles Active (%),L2: D Sectors (%),L1: Lsuin Requests (%),L1: Data Bank Reads (%),L2: D Sectors Fill Device (%),L1: Data Bank Writes (%),L1: Texin Sm2tex Req Cycles Active (%),L1: Data Pipe Tex Wavefronts (%),L1: F Wavefronts (%),L1: Tex Writeback Active (%),L2: D Atomic Input Cycles Active (%),L2: D Sectors Fill Sysmem (%),sm__sass_thread_inst_executed_op_dfma_pred_on.sum.peak_sustained (inst/cycle),sm__sass_thread_inst_executed_op_ffma_pred_on.sum.peak_sustained (inst/cycle),smsp__sass_thread_inst_executed_op_dfma_pred_on.sum.per_cycle_elapsed (inst/cycle),smsp__sass_thread_inst_executed_op_ffma_pred_on.sum.per_cycle_elapsed (inst/cycle),derived__sm__sass_thread_inst_executed_op_ffma_pred_on_x2 (inst),sm__cycles_elapsed.avg.per_second (cycle/second),dram__bytes.sum.peak_sustained (byte/cycle),dram__cycles_elapsed.avg.per_second (cycle/second),derived__sm__sass_thread_inst_executed_op_dfma_pred_on_x2 (inst),sm__cycles_elapsed.avg.per_second (cycle/second),dram__bytes.sum.peak_sustained (byte/cycle),dram__cycles_elapsed.avg.per_second (cycle/second),derived__smsp__sass_thread_inst_executed_op_ffma_pred_on_x2 (inst),smsp__cycles_elapsed.avg.per_second (cycle/second),smsp__sass_thread_inst_executed_op_fadd_pred_on.sum.per_cycle_elapsed (inst/cycle),smsp__sass_thread_inst_executed_op_fmul_pred_on.sum.per_cycle_elapsed (inst/cycle),dram__bytes.sum.per_second (byte/second),derived__smsp__sass_thread_inst_executed_op_dfma_pred_on_x2 (inst),smsp__cycles_elapsed.avg.per_second (cycle/second),smsp__sass_thread_inst_executed_op_dadd_pred_on.sum.per_cycle_elapsed (inst/cycle),smsp__sass_thread_inst_executed_op_dmul_pred_on.sum.per_cycle_elapsed (inst/cycle),dram__bytes.sum.per_second (byte/second),Executed Ipc Active (inst/cycle),Executed Ipc Elapsed (inst/cycle),Issue Slots Busy (%),Issued Ipc Active (inst/cycle),sm__inst_issued.max.pct_of_peak_sustained_active (%),SM Busy (%),Tensor (All) (%),Tensor (FP) (%),ALU (%),FMA (%),FP64 (%),Tensor (INT) (%),Tensor (FP) (%),LSU (%),ALU (%),Uniform (%),ADU (%),FMA (%),CBU (%),FMA (FP16) (%),FP64 (DMMA) (%),FP64 (FP64) (%),Tensor (INT) (%),TEX (%),XU (%),Memory Throughput (byte/second),Mem Busy (%),Max Bandwidth (%),L1/TEX Hit Rate (%),L2 Compression Success Rate (%),NaN,L2 Hit Rate (%),Mem Pipes Busy (%),lts__average_gcomp_input_sector_success_rate.pct (%),lts__t_sectors_srcunit_tex.avg.pct_of_peak_sustained_elapsed (%),lts__t_sectors_srcunit_tex_aperture_peer_lookup_miss.sum (sector),lts__t_sectors_srcunit_tex_aperture_sysmem_lookup_miss.sum (sector),lts__t_sectors_srcunit_tex_lookup_miss.sum (sector),dram__bytes_read.sum (byte),dram__bytes_read.sum.pct_of_peak_sustained_elapsed (%),dram__bytes_read.sum.per_second (byte/second),dram__bytes_write.sum (byte),dram__bytes_write.sum.pct_of_peak_sustained_elapsed (%),dram__bytes_writ

In [32]:
df = df.loc[:,~df.columns.duplicated()].copy()
df = df.loc[:, df.columns.notna()]

In [37]:
### important columns for now
key_cols = ['M', 'K', 'N', 'Duration (nsecond)', 'Compute (SM) Throughput (%)', 'Memory Throughput (%)', 'Memory Throughput (byte/second)', 'DRAM Throughput (%)', 'Mem Busy (%)', 'Max Bandwidth (%)', 'Theoretical Occupancy (%)', 'Achieved Occupancy (%)', 
            'Elapsed Cycles (cycle)', 'SM Active Cycles (cycle)', 'SM: Pipe Tensor Cycles Active (%)',
            'Executed Ipc Active (inst/cycle)', 'Executed Ipc Elapsed (inst/cycle)', 'Issue Slots Busy (%)', 'Issued Ipc Active (inst/cycle)', 'SM Busy (%)', 'Tensor (All) (%)', 'ALU (%)', 'FMA (%)', 'FMA (FP16) (%)', 'FP64 (%)', 'FP64 (DMMA) (%)',  
            'L1/TEX Hit Rate (%)', 'L2 Hit Rate (%)', 'Mem Pipes Busy (%)',
            'Active Warps Per Scheduler (warp)', 'Eligible Warps Per Scheduler (warp)', 'Theoretical Warps Per Scheduler (warp)', 'GPU Maximum Warps Per Scheduler (warp)', 'Active Warps Per Scheduler (warp)', 'Eligible Warps Per Scheduler (warp)', 'Warp Cycles Per Issued Instruction (cycle)', 'Warp Cycles Per Executed Instruction (cycle)',
            'Avg. Executed Instructions Per Scheduler (inst)', 'Executed Instructions (inst)', 'Avg. Issued Instructions Per Scheduler (inst)', 'Issued Instructions (inst)', 'Registers Per Thread (register/thread)', 'Shared Memory Per Block (byte/block)', 'Shared Memory Configuration Size (byte)', 'Driver Shared Memory Per Block (byte/block)', 'Dynamic Shared Memory Per Block (byte/block)', 'Static Shared Memory Per Block (byte/block)', 'Threads (thread)', 'Avg. Threads Executed (thread)', 'Avg. Predicated-On Threads Executed (thread)',
            'Theoretical Active Warps per SM (warp)', 'Achieved Active Warps Per SM (warp)',
            'L1 Wavefronts Shared Excessive (byte)', 'L2 Theoretical Sectors Global Excessive (byte)', 'Branch Instructions Ratio (%)',
            'Kernel Name', 'Block Size', 'Grid Size']

In [38]:
df_key = df[key_cols]

In [39]:
df_key

,M,K,N,Duration (nsecond),Compute (SM) Throughput (%),Memory Throughput (%),Memory Throughput (byte/second),DRAM Throughput (%),Mem Busy (%),Max Bandwidth (%),Theoretical Occupancy (%),Achieved Occupancy (%),Elapsed Cycles (cycle),SM Active Cycles (cycle),SM: Pipe Tensor Cycles Active (%),Executed Ipc Active (inst/cycle),Executed Ipc Elapsed (inst/cycle),Issue Slots Busy (%),Issued Ipc Active (inst/cycle),SM Busy (%),Tensor (All) (%),ALU (%),FMA (%),FMA (FP16) (%),FP64 (%),FP64 (DMMA) (%),L1/TEX Hit Rate (%),L2 Hit Rate (%),Mem Pipes Busy (%),Active Warps Per Scheduler (warp),Eligible Warps Per Scheduler (warp),Theoretical Warps Per Scheduler (warp),GPU Maximum Warps Per Scheduler (warp),Active Warps Per Scheduler (warp),Eligible Warps Per Scheduler (warp),Warp Cycles Per Issued Instruction (cycle),Warp Cycles Per Executed Instruction (cycle),Avg. Executed Instructions Per Scheduler (inst),Executed Instructions (inst),Avg. Issued Instructions Per Scheduler (inst),Issued Instructions (inst),Registers Per Thread (register/thread),Shared Memory Per Block (byte/block),Shared Memory Configuration Size (byte),Driver Shared Memory Per Block (byte/block),Dynamic Shared Memory Per Block (byte/block),Static Shared Memory Per Block (byte/block),Threads (thread),Avg. Threads Executed (thread),Avg. Predicated-On Threads Executed (thread),Theoretical Active Warps per SM (warp),Achieved Active Warps Per SM (warp),L1 Wavefronts Shared Excessive (byte),L2 Theoretical Sectors Global Excessive (byte),Branch Instructions Ratio (%),Kernel Name,Block Size,Grid Size
0,128,128,128,"6,656",0.82,3.11,"22,596,153,846.15",2.79,3.11,2.79,16.67,8.32,"8,275",310.52,0.82,0.80,0.03,20.52,0.82,20.52,17.22,16.09,2.61,0,0,0,0,80.42,0.82,0.99,0.21,2,12,0.99,0.21,4.80,4.90,62.49,"20,496",63.71,"20,896",100,"50,176","102,400","1,024","49,152",0,512,32,30,8,3.99,"1,664",0,0.01,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 1)"
1,128,256,128,"8,480",1.03,4.04,"34,173,584,905.66",4.04,3.86,4.04,8.33,8.29,"10,969",437.44,1.03,0.83,0.03,21.15,0.85,22.84,22.84,18.47,1.64,0,0,0,0,59.05,1.03,1.00,0.21,1,12,1.00,0.21,4.75,4.82,91.12,"29,888",92.54,"30,352",102,"99,328","102,400","1,024","98,304",0,512,32,30,4,3.98,0,0,0.01,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 1)"
2,128,512,128,"6,048",5.13,14.78,"89,862,433,862.43",11.06,14.78,11.06,16.67,8.08,"7,470","2,088.60",5.13,0.72,0.20,18.35,0.73,18.35,15.92,9.57,3.06,0,0,0,0,83.72,5.06,0.99,0.19,2,12,0.99,0.19,5.29,5.42,373.46,"122,496",383.22,"125,696",100,"50,176","102,400","1,024","49,152",0,"4,096",32,30,8,3.88,"8,192",0,0.01,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 8)"
3,128,1024,128,"8,448",4.54,15.46,"126,560,606,060.61",15.46,15.02,15.46,8.33,8.23,"10,535","2,238.06",4.54,0.74,0.16,18.69,0.75,18.69,17.86,16.59,2.49,0,0,0,0,77.14,4.54,0.96,0.18,1,12,0.96,0.18,5.27,5.35,411.61,"135,008",418.33,"137,213",157,"74,752","102,400","1,024","73,728",0,"2,816",32,30,4,3.95,"11,648",0,0.01,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(2, 1, 11)"
4,128,2048,128,"8,704",7.73,28.41,"242,250,000,000",28.41,28.32,28.41,16.67,8.29,"11,195","3,569.02",7.73,0.85,0.27,21.63,0.87,22.39,22.39,17.71,1.92,0,0,0,0,77.74,7.73,0.99,0.22,2,12,0.99,0.22,4.57,4.63,762.15,"249,984",771.90,"253,184",100,"50,176","102,400","1,024","49,152",0,"4,096",32,29,8,3.98,"21,504",0,0.01,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 8)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,65536,1024,16384,"41,837,120",44.79,63.61,"375,520,343,274.11",41.11,63.61,58.72,16.67,16.64,"58,479,339","58,197,220.29",44.79,0.63,0.63,15.73,0.63,45.00,45.00,10.35,0.70,0,0,0,0,89.25,17.80,2.00,0.21,2,12,2.00,0.21,12.68,12.68,"9,155,059.51","3,002,859,520","9,1